In [52]:
import math
from shapely.geometry import Point
import folium
import pandas as pd
import geopandas as gpd
import requests
import os
from dotenv import load_dotenv
load_dotenv(".env")
from tqdm import tqdm

In [53]:
def search_locations(lat, lng, radius):
    """
    指定した緯度・経度を中心に、半径radiusメートル以内で結果を返す関数
    :param lat: 緯度
    :param lng: 経度
    :param radius: 検索半径（メートル）
    """
    API_KEY = os.getenv("API_KEY")
    API_URL = os.getenv("API_URL")

    # API Request

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.location,places.id",
    }

    data = {
        "includedTypes": ["restaurant"],
        "maxResultCount": 10,
        "locationRestriction": {"circle": {"center": {"latitude": lat, "longitude": lng}, "radius": radius}},
        "languageCode": "ja",
    }

    response = requests.post(API_URL, headers=headers, json=data)
    res = response.json()
    
    # Convert to GeoDataFrame
    
    df = pd.DataFrame()
    
    if "places" not in res:
        return df
    for place in res["places"]:
        new_df = gpd.GeoDataFrame(
            [
                {
                    "name": place["displayName"]["text"],
                    "geometry": Point(place["location"]["longitude"], place["location"]["latitude"]),
                    "place_id": place["id"],
                }
            ]
        )
        df = pd.concat([df, new_df])    
        
    return df


def search_locations_from_multi_area(grid_centers, radius_m):
    """
    複数の地点を指定して、それぞれの地点を中心に、半径radiusメートル以内で結果を返す関数
    :param grid_centers: (lat, lon)のコラムを持つGeoDataFrame
    :param radius: 検索半径（メートル）
    """
    results = gpd.GeoDataFrame()
    for i in tqdm(range(len(grid_centers))):
        lat = grid_centers.loc[i, "lat"]
        lon = grid_centers.loc[i, "lon"]
        res = search_locations(lat, lon, radius_m)
        results = pd.concat([results, res])
        
    return results

In [54]:
# gridデータの読み込み
tokuyama_grid = gpd.read_file("./data/tokuyama-grid.csv")

In [55]:
# それぞれのgridから飲食店を検索
radius = os.getenv("RADIUS")
results = search_locations_from_multi_area(tokuyama_grid, radius)

100%|██████████| 760/760 [05:37<00:00,  2.25it/s]


In [59]:
results_df = gpd.GeoDataFrame(results)
results_df.to_file("./data/results.geojson", driver="GeoJSON")

/usr/local/lib/python3.12/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
